In [ ]:
pip install kafka-python

In [1]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json

In [10]:
producer = KafkaProducer(bootstrap_servers=['{your-port}:9092'], #change ip here
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [3]:
#producer.send('testing1', value={'hope':'never'})

In [11]:
df = pd.read_csv("data_stocks.csv")

In [12]:
df.head(5)

,Datetime,Adj Close_AAPL,Adj Close_IBM,Adj Close_TSLA,Close_AAPL,Close_IBM,Close_TSLA,High_AAPL,High_IBM,High_TSLA,Low_AAPL,Low_IBM,Low_TSLA,Open_AAPL,Open_IBM,Open_TSLA,Volume_AAPL,Volume_IBM,Volume_TSLA
0,2024-06-17 09:30:00-04:00,212.993607,168.130005,180.199707,212.993607,168.130005,180.199707,214.440002,168.759995,180.300003,212.960007,167.820007,177.000000,213.300003,168.759995,177.949997,5198111,139002.0,5019092
1,2024-06-17 09:35:00-04:00,213.315994,167.544998,180.167206,213.315994,167.544998,180.167206,213.705002,168.250000,180.940002,212.720001,167.500000,179.699997,212.994995,168.154999,180.195007,1672453,41521.0,2596640
2,2024-06-17 09:40:00-04:00,213.723999,168.020004,180.408005,213.723999,168.020004,180.408005,214.080002,168.080002,180.739899,213.250000,167.500000,179.419998,213.330002,167.535004,180.179993,1409081,35748.0,2257603
3,2024-06-17 09:45:00-04:00,213.570007,168.460007,180.869995,213.570007,168.460007,180.869995,214.800003,168.460007,181.610001,213.229996,168.000900,180.414993,213.720001,168.020004,180.419998,2134755,30790.0,2477614
4,2024-06-17 09:50:00-04:00,213.463104,168.300003,181.065002,213.463104,168.300003,181.065002,213.589996,168.699997,181.490005,212.960007,168.300003,180.440002,213.559998,168.520004,180.860001,1350967,20405.0,1419959


In [23]:
#total_data = df.size
#print(f"Total data: {total_data}")


In [22]:
#df.sample(1).to_dict(orient="records")[0]

In [ ]:
#from time import sleep, time

# Batch sending
#batch_size = 10 # Number of samples shipped at once
#interval = 1 # Send time interval
#max_duration = 60 # Maximum duration in seconds

#start_time = time()

#while time() - start_time < max_duration:
#    dict_stocks = df.sample(batch_size).to_dict(orient="records")
#    for dict_stock in dict_stocks:
#        producer.send('testing3', value=dict_stock)
#    sleep(interval)

In [13]:
# Batch sending
batch_size = 10 # Number of samples shipped at once
interval = 1 # Send time interval

# Use iterrows() to iterate over DataFrame rows
iterator = df.iterrows()
batch = []

while True:
    try:
        for _ in range(batch_size):
            _, row = next(iterator)
            batch.append(row.to_dict())

        for dict_stock in batch:
            producer.send('testing2', value=dict_stock)
        
        # Flush after sending a batch
        producer.flush()
        
        batch = []
        sleep(interval)
    except StopIteration:
        # Send remaining batch if any
        if batch:
            for dict_stock in batch:
                producer.send('testing2', value=dict_stock)
            producer.flush()
        break

# Close the producer
producer.close()

In [7]:
producer.flush() #clear data from kafka server